# 🚀 Customize and Deploy `deepseek-ai/DeepSeek-R1-0528` on Amazon SageMaker AI

In this notebook, we explore **DeepSeek-R1-0528**, a cutting-edge reasoning model from DeepSeek AI. You'll learn how to fine-tune it on reasoning datasets, evaluate its mathematical and logical capabilities, and deploy it using SageMaker for advanced reasoning tasks.

## What is DeepSeek-R1-0528?
DeepSeek-R1-0528 is part of DeepSeek's R1 series, specifically designed for advanced reasoning capabilities. This model represents a significant advancement in AI reasoning, combining deep learning with sophisticated reasoning mechanisms to tackle complex mathematical, logical, and analytical problems. It builds upon DeepSeek's expertise in creating efficient and powerful language models.  
🔗 Model card: [deepseek-ai/DeepSeek-R1-0528 on Hugging Face](https://huggingface.co/deepseek-ai/DeepSeek-R1-0528)

## Key Specifications
| Feature | Details |
|---|---|
| **Parameters** | Multi-billion parameter architecture optimized for reasoning |
| **Architecture** | Advanced Transformer with specialized reasoning modules |
| **Context Length** | Extended context window for complex reasoning chains |
| **Modalities** | Text-in / Text-out with focus on reasoning tasks |
| **License** | Check model card for specific licensing terms |
| **Release Date** | May 28th release (0528) |

## Benchmarks & Behavior
- Exceptional performance on **mathematical reasoning, logical inference, and complex problem-solving** benchmarks.  
- Designed to excel at **multi-step reasoning tasks** with clear chain-of-thought capabilities.  
- Strong performance on competition mathematics, coding challenges, and analytical reasoning tasks.  
- Optimized for **step-by-step problem decomposition** and systematic solution approaches.
    
---

In [ ]:
%pip install -Uq sagemaker datasets

In [ ]:
import os
import boto3
import sagemaker

In [ ]:
region = boto3.Session().region_name

sess = sagemaker.Session(boto3.Session(region_name=region))

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

In [ ]:
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Data Preparation for Supervised Fine-tuning

### [NuminaMath-CoT](https://huggingface.co/datasets/AI-MO/NuminaMath-CoT)

**NuminaMath-CoT** is a large-scale dataset of **~860,000+ math competition question-solution pairs**, designed to support chain-of-thought reasoning in mathematical problem solving.

**Data Format & Structure**:
- Each example is a question followed by a solution; the solution is formatted with detailed **Chain-of-Thought (CoT)** reasoning.  
- The data sources include *Chinese high school math exercises*, *US and international mathematics competition problems*, *online test-papers PDFs*, and *math discussion forums*.  
- Preprocessing includes OCR from PDFs, segmentation to extract problem-solution pairs, translation into English, alignment into CoT style, and formatting of final answers.  

**License**: Released under the **Apache-2.0** license.  

**Applications**:

This dataset is useful for training and evaluating models on tasks including:  
- Complex math problem solving with reasoning steps (algebra, geometry, number theory, etc.)  
- Benchmarking chain-of-thought performance of LLMs on competition-level math tasks  
- Educational tools and tutoring systems that require explainable math solutions  
- Fine-tuning models to improve consistency, reasoning depth, and accuracy in mathematical domains  

In [ ]:
import os
import json
import pprint
from tqdm import tqdm
from datasets import load_dataset

In [ ]:
dataset_parent_path = os.path.join(os.getcwd(), "tmp_cache_local_dataset")
os.makedirs(dataset_parent_path, exist_ok=True)

**Preparing Your Dataset in `messages` format**

This section walks you through creating a conversation-style dataset—the required `messages` format—for directly training LLMs using SageMaker AI.

**What Is the `messages` Format?**

The `messages` format structures instances as chat-like exchanges, wrapping each conversation turn into a role-labeled JSON array. It’s widely used by frameworks like TRL.

Example entry:

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "How do I bake sourdough?" },
    { "role": "assistant", "content": "First, you need to create a starter by..." }
  ]
}


In [ ]:
dataset_name = "AI-MO/NuminaMath-CoT"
dataset = load_dataset(dataset_name, split="train[:1000]")

In [ ]:
pprint.pp(dataset[0])

In [ ]:
print(f"total number of fine-tunable samples: {len(dataset)}")

In [ ]:
def convert_to_messages_reasoning(row):
    system_content = "You are a mathematical reasoning assistant. Read the problem, restate the key givens and goal, then solve step-by-step with clear algebra (use LaTeX), keeping exact arithmetic (fractions/surds) and justifying each transformation (e.g., equal differences for arithmetic sequences). Verify any domain or extraneous-solution constraints, and present the final simplified answer concisely on the last line."
    
    messages_user_row = row["messages"][0]
    assert messages_user_row["role"] == "user", f"user row unmatched"
    user_content = messages_user_row["content"]
    
    messages_assistant_row = row["messages"][1]
    assert messages_assistant_row["role"] == "assistant", f"assistant row unmatched"
    assistant_content = messages_assistant_row["content"]

    think_block = f"<think>{row['solution']}</think>"
    
    return {
        "messages": [
            { "role": "system", "content": system_content},
            { "role": "user", "content": user_content },
            { "role": "assistant", "content": f"{think_block}\n\n{assistant_content}" }
        ]
    }
    
    
dataset = dataset.map(convert_to_messages_reasoning, remove_columns=dataset.column_names)

In [ ]:
dataset_filename = os.path.join(dataset_parent_path, f"{dataset_name.replace('/', '--').replace('.', '-')}.jsonl")
dataset.to_json(dataset_filename, lines=True)

#### Upload file to S3

In [ ]:
from sagemaker.s3 import S3Uploader

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/dataset"

uploaded_s3_uri = S3Uploader.upload(
    local_path=dataset_filename,
    desired_s3_uri=data_s3_uri
)
print(f"Uploaded {dataset_filename} to > {uploaded_s3_uri}")

## Fine-Tune LLMs using SageMaker AI

In [ ]:
import time
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.modules.configs import InputData
from sagemaker.modules.train import ModelTrainer
from getpass import getpass
import yaml
from jinja2 import Template

In [ ]:
MODEL_ID = "deepseek-ai/DeepSeek-R1-0528"

In [ ]:
hf_token = getpass()

### Train FM using SageMaker AI `ModelTrainer`

---
**Observability**: SageMaker AI has [SageMaker MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html) which enables you to accelerate generative AI by making it easier to track experiments and monitor performance of models and AI applications using a single tool.

You can choose to include MLflow as a part of your training workflow to track your model fine-tuning metrics in realtime by simply specifying a **mlflow** tracking arn.

Optionally you can also report to : **tensorboard**, **wandb**.

In [ ]:
MLFLOW_TRACKING_SERVER_ARN = "arn:aws:sagemaker:us-east-1:811828458885:mlflow-tracking-server/mlflow-demos"

if MLFLOW_TRACKING_SERVER_ARN:
    reports_to = "mlflow"
else:
    reports_to = "tensorboard"

In [ ]:
job_name = MODEL_ID.replace('/', '--').replace('.', '-')

In [ ]:
if MLFLOW_TRACKING_SERVER_ARN:
    training_env = {
        # mlflow tracking metrics
        "MLFLOW_EXPERIMENT_NAME": f"{job_name}-exp",
        "MLFLOW_TAGS": json.dumps(
            {
                "source.job": "sm-training-jobs", 
                "source.type": "sft", 
                "source.framework": "pytorch"
            }
        ),
        "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_SERVER_ARN,
        "MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING": "true",
        # non tracking metrics - enabled
        "HF_TOKEN": hf_token,
        "FI_EFA_USE_DEVICE_RDMA": "1",
        "NCCL_DEBUG": "INFO",
        "NCCL_SOCKET_IFNAME": "eth0",
        "FI_PROVIDER": "efa",
        "NCCL_PROTO": "simple",
        "NCCL_NET_GDR_LEVEL": "5"
    }
else:
    training_env = {
        # non tracking metrics
        "HF_TOKEN": hf_token,
        "FI_EFA_USE_DEVICE_RDMA": "1",
        "NCCL_DEBUG": "INFO",
        "NCCL_SOCKET_IFNAME": "eth0",
        "FI_PROVIDER": "efa",
        "NCCL_PROTO": "simple",
        "NCCL_NET_GDR_LEVEL": "5"
    }

#### Training strategy - Choose between: `PeFT`/`Spectrum`/`Full-Finetuning`

Here we create a measured mapping of strategy to instance.

In [ ]:
%%writefile sagemaker_code/requirements.txt
transformers==4.57.1
peft==0.17.0
accelerate==1.11.0
bitsandbytes==0.46.1
datasets==4.0.0
deepspeed==0.17.5
hf-transfer==0.1.8
hf_xet
liger-kernel==0.6.1
lm-eval[api]==0.4.9
kernels>=0.9.0
mlflow
Pillow
safetensors>=0.6.2
sagemaker==2.251.1
sagemaker-mlflow==0.1.0
sentencepiece==0.2.0
tokenizers>=0.21.4
triton
trl==0.21.0
tensorboard
psutil
py7zr
git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels
vllm==0.10.1
poetry
yq
psutil
nvidia-ml-py
pyrsmi

In [ ]:
# For PeFT
args = [
    "--config",
    "hf_recipes/deepseek-ai/DeepSeek-R1-0528--vanilla-peft-qlora.yaml",
    # "--run-eval" # enable this for small models to run eval + tune
]
training_instance_type = "ml.p5en.48xlarge"
training_instance_count = 1

## For Spectrum
# args = [
#     "--config",
#     "hf_recipes/deepseek-ai/DeepSeek-R1-0528--vanilla-spectrum.yaml",
#     # "--run-eval" # enable this for small models if you're looking to bundle eval with fine-tuning
# ]
# training_instance_type = "ml.p5en.48xlarge"
# training_instance_count = 1

## For Full-Finetuning
# args = [
#     "--config",
#     "hf_recipes/deepseek-ai/DeepSeek-R1-0528--vanilla-full.yaml",,
#     # "--run-eval" # enable this for small models if you're looking to bundle eval with fine-tuning
# ]
# training_instance_type = "ml.p5en.48xlarge"
# training_instance_count = 1


In [ ]:
pytorch_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sess.boto_session.region_name,
    version="2.8.0",
    instance_type=training_instance_type,
    image_scope="training",
)
print(f"Using image: {pytorch_image_uri}")

In [ ]:
source_code = SourceCode(
    source_dir="./sagemaker_code",
    command=f"bash sm_accelerate_train.sh {' '.join(args)}",
)

compute_configs = Compute(
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    keep_alive_period_in_seconds=1800,
    volume_size_in_gb=300
)

base_job_name = f"{job_name}-finetune"
output_path = f"s3://{sess.default_bucket()}/{base_job_name}"

model_trainer = ModelTrainer(
    training_image=pytorch_image_uri,
    source_code=source_code,
    base_job_name=base_job_name,
    compute=compute_configs,
    stopping_condition=StoppingCondition(max_runtime_in_seconds=18000),
    output_data_config=OutputDataConfig(
        s3_output_path=output_path,
    ),
    checkpoint_config=CheckpointConfig(
        s3_uri=os.path.join(
            output_path,
            dataset_name.replace('/', '--').replace('.', '-'), 
            job_name,
            "checkpoints"
        ), 
        local_path="/opt/ml/checkpoints"
    ),
    role=role,
    environment=training_env
)

In [ ]:
model_trainer.train(
    input_data_config=[
        InputData(
            channel_name="training",
            data_source=uploaded_s3_uri,  
        )
    ], 
    wait=False
)